In [30]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob
from textblob import Word
import re
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

In [74]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [64]:
print ("PART 4")

PART 4


In [2]:
data = pd.read_csv("spam.csv",encoding='latin-1')

In [3]:
data.shape

(1000, 5)

In [4]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

In [5]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
names = pd.read_csv("names.csv", delimiter='\t')
names = pd.DataFrame(names.values, columns=['names'])
names = names['names'].str.split(" ", n = 1, expand = True) 
names = names[0].tolist()

In [7]:
regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

In [8]:
def checkName(text):
    text = text.values[0]
    words = text.split(' ')
    count = 0
    for each in words:
        if each.upper() in names:
            count+=1
    return count

In [9]:
def validURL(text):
    text = text.values[0]
    words = text.split(' ')
    count = 0
    for each in words:
        var = re.match(regex, each)
        if var is not None:
            if var == True:
                count+=1
    return count

In [10]:
def getURLNum(text):
    text = text.values[0]
    words = text.split(' ')
    count = 0
    for each in words:
        if len(each) > 3 and each[0] == 'h' and each[1] == 't' and each[2] =='t' and each[3] == 'p':
            count+=1
    return count

In [11]:
def checkWord(text):
    text = text.values[0]
    words = text.split(' ')
    count = 0
    for each in words:
        w = Word(each)
        var = w.spellcheck()
        if var[0][1] <= 0.5:
            count+=1
    return count

In [12]:
def countWords(text):
    text = text.values[0]
    words = text.split(' ')
    count = len(words)
    return count

In [13]:
def pronouns(text):
    text = text.values[0]
    wiki = TextBlob(text)
    return len(wiki.tags)

In [14]:
newList = []
k = 0
while k < data.shape[0]:
    newList.append(getURLNum(data['v2'][data['v2'].index==k]))
    k+=1

In [15]:
data['numOfUrl'] = newList

In [16]:
newList = []
k = 0
while k < data.shape[0]:
    newList.append(checkWord(data['v2'][data['v2'].index==k]))
    k+=1

In [17]:
data['spellingMistakes'] = newList

In [18]:
newList = []
k = 0
while k < data.shape[0]:
    newList.append(countWords(data['v2'][data['v2'].index==k]))
    k+=1

In [19]:
data['countOfWords'] = newList

In [20]:
newList = []
k = 0
while k < data.shape[0]:
    newList.append(checkName(data['v2'][data['v2'].index==k]))
    k+=1

In [21]:
data['namesCount'] = newList

In [22]:
newList = []
k = 0
while k < data.shape[0]:
    newList.append(validURL(data['v2'][data['v2'].index==k]))
    k+=1

In [23]:
data['validURLCount'] = newList

In [24]:
newList = []
k = 0
while k < data.shape[0]:
    newList.append(pronouns(data['v2'][data['v2'].index==k]))
    k+=1

In [25]:
data['numProunous'] = newList

In [27]:
data.pop('v2')
label = data.pop('v1')

In [40]:
print ("TRAINING WITH NEW FEATURES")

TRAINING WITH NEW FEATURES


In [33]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)

In [38]:
n = SVC()
n.fit(X_train, y_train)
prediction = n.predict(X_test)
print ("SVM Accuracy ",accuracy_score(y_test, prediction))

SVM Accuracy  0.9272727272727272


In [63]:
print ("PART 5")

PART 5


In [44]:
print ("TRAINING WITH TF.IDF FEATURES and PCA 10")

TRAINING WITH TF.IDF FEATURES and PCA 10


In [45]:
fullData = pd.read_csv("spam.csv",encoding='latin-1')

In [46]:
fullData = fullData.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

In [52]:
corpus = []
k = 0
while k < fullData.shape[0]:
    corpus.append(fullData['v2'].values.tolist()[k])
    k+=1

In [53]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [57]:
X = X.toarray()

In [58]:
n = PCA(10)
pcaX = n.fit_transform(X)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(pcaX, label, test_size=0.33, random_state=42)

In [61]:
n = SVC()
n.fit(X_train, y_train)
prediction = n.predict(X_test)
print ("SVM Accuracy ",accuracy_score(y_test, prediction))

SVM Accuracy  0.8515151515151516
